In [1]:
from mia_models import train_target_model, load_shadow_data, train_shadow_models
from mia_models import prepare_attack_test_data, prep_validation_data, prep_attack_train_data
from mia_models import shokri_attack, prety_print_result
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

2023-06-06 12:38:09.057322: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-06 12:38:09.057460: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [32]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [2]:
dataset_name = 'location'
train_size = 100
attack_test_size = 50
test_start = 100

loc_data = pd.read_csv('data/Location/bangkok', na_values=["?"], header=None)
target_dataset = loc_data.sample(n = 1200, replace = False)
df_rest = loc_data.loc[~loc_data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 2000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [3]:
# trian target model
per_class_sample=40
channel=1   
epochs=100
n_class = 31
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)

2023-06-06 12:38:22.491410: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-06 12:38:22.491560: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-06 12:38:22.491668: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-BP5MTC3R): /proc/driver/nvidia/version does not exist
2023-06-06 12:38:22.492408: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



 Target Train acc :  100.0 Target Test acc :  23.545454442501068


In [ ]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 1200

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

In [69]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list, mpred, nmpred = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

2/2 [==============================] - 0s 3ms/step
1
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 20ms/step
3
1/1 [==============================] - 0s 21ms/step
4
1/1 [==============================] - 0s 28ms/step
5
6
1/1 [==============================] - 0s 31ms/step
7
1/1 [==============================] - 0s 28ms/step
8
1/1 [==============================] - 0s 26ms/step
9
10
1/1 [==============================] - 0s 31ms/step
11
12
13
1/1 [==============================] - 0s 24ms/step
14
1/1 [==============================] - 0s 22ms/step
15
1/1 [==============================] - 0s 26ms/step
16
17
1/1 [==============================] - 0s 24ms/step
18
19
1/1 [==============================] - 0s 25ms/step
20
1/1 [==============================] - 0s 29ms/step
21
22
1/1 [==============================] - 0s 26ms/step
23
1/1 [==============================] - 0s 27ms/step
24
25
26
1/1 [==============================] - 0s 21ms/s

In [70]:
mpredi = np.array([item for sublist in mpred for item in sublist])
len(mpredi)

39

In [71]:
nmpredi = np.array([item for sublist in nmpred for item in sublist])

In [72]:
df = pd.DataFrame(mpredi)
df['memberstatus'] = 'member'
df_nm = pd.DataFrame(nmpredi)
df_nm['memberstatus'] = 'nonmember'
df_nm
frames = [df, df_nm]
df_org = pd.concat(frames)
df_org

,0,1,memberstatus
0,0.157652,0.842348,member
1,0.144889,0.855111,member
2,0.193260,0.806740,member
3,0.165536,0.834464,member
4,0.171518,0.828482,member
...,...,...,...
40,0.972808,0.027192,nonmember
41,0.999956,0.000044,nonmember
42,0.999977,0.000023,nonmember
43,0.999996,0.000004,nonmember


In [44]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [5]:
# synthetic dataset
train_size = 100
attack_test_size = 50
test_start = 100
org_dataset = pd.read_csv('data/Location/bangkok', na_values=["?"], header=None)
target_dataset = pd.read_csv('data/Location/loc_sds_cac_200.csv', na_values=["?"], header=None).sample(n = 150, replace = False)
df = org_dataset.iloc[2500:,:]
# shadow_dataset = df.sample(n = 1900, replace = False)
# df_rest = df.loc[~df.index.isin(shadow_dataset.index)]
# attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
# attack_test_members = org_dataset.iloc[:200,:]
# attack_test_members = attack_test_members.sample(n=attack_test_size, replace=False)

In [6]:
# trian target model
per_class_sample=40
channel=1   
epochs=100
n_class = 31
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)


 Target Train acc :  100.0 Target Test acc :  62.00000047683716


In [75]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 1200

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  100.0 Shadow Test acc :  48.88888895511627
12/12 [==============================] - 0s 2ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  100.0 Shadow Test acc :  46.11110985279083
12/12 [==============================] - 0s 3ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  100.0 Shadow Test acc :  46.11110985279083
12/12 [==============================] - 0s 3ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  100.0 Shadow Test acc :  44.72222328186035
12/12 [==============================] - 0s 4ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  100.0 Shadow Test acc :  45.83333432674408
12/12 [==============================] - 0s 4ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  91.3095235824585 Shadow Test acc :  44.16666626930237
12/12 [==============================] - 0s 3ms/step
Shadow Model  6
Training shadow model 6
Shadow Train 

In [92]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list, mpred_syn, nmpred_syn = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

2/2 [==============================] - 0s 4ms/step
1
1/1 [==============================] - 0s 43ms/step
2
1/1 [==============================] - 0s 26ms/step
3
1/1 [==============================] - 0s 27ms/step
4
1/1 [==============================] - 0s 27ms/step
5
6
1/1 [==============================] - 0s 31ms/step
7
1/1 [==============================] - 0s 25ms/step
8
1/1 [==============================] - 0s 29ms/step
9
10
1/1 [==============================] - 0s 26ms/step
11
12
13
1/1 [==============================] - 0s 31ms/step
14
1/1 [==============================] - 0s 24ms/step
15
1/1 [==============================] - 0s 24ms/step
16
17
1/1 [==============================] - 0s 25ms/step
18
19
1/1 [==============================] - 0s 25ms/step
20
1/1 [==============================] - 0s 25ms/step
21
22
1/1 [==============================] - 0s 24ms/step
23
1/1 [==============================] - 0s 24ms/step
24
25
26
1/1 [==============================] - 0s 25ms/s

In [93]:
mpredi_syn = np.array([item for sublist in mpred_syn for item in sublist])
nmpredi_syn = np.array([item for sublist in nmpred_syn for item in sublist])
mpredi_syn

array([[9.99905348e-01, 9.46597283e-05],
       [9.97955084e-01, 2.04488868e-03],
       [9.99169350e-01, 8.30700621e-04],
       [5.17923176e-01, 4.82076824e-01],
       [2.83808529e-01, 7.16191471e-01],
       [9.97881114e-01, 2.11884454e-03],
       [9.99923825e-01, 7.61248302e-05],
       [9.99999404e-01, 5.68642918e-07],
       [9.47402656e-01, 5.25974147e-02],
       [9.99322772e-01, 6.77215809e-04],
       [9.96496856e-01, 3.50315729e-03],
       [9.67696309e-01, 3.23036425e-02],
       [9.99760330e-01, 2.39625850e-04],
       [9.96879458e-01, 3.12055438e-03],
       [9.97849464e-01, 2.15049158e-03],
       [9.92586434e-01, 7.41357869e-03],
       [9.93614793e-01, 6.38518902e-03],
       [8.73456478e-01, 1.26543567e-01],
       [9.93828475e-01, 6.17149798e-03],
       [9.91454959e-01, 8.54506996e-03],
       [8.74830663e-01, 1.25169367e-01],
       [9.84921038e-01, 1.50789432e-02],
       [9.98017550e-01, 1.98242161e-03],
       [8.92164052e-01, 1.07835919e-01],
       [9.958508

In [99]:
df_syn_m = pd.DataFrame(mpredi_syn)
df_syn_m['memberstatus'] = 'member'
df_syn_nm = pd.DataFrame(nmpredi_syn)
df_syn_nm['memberstatus'] = 'nonmember'
frames_syn = [df_syn_m, df_syn_nm]
df_syn = pd.concat(frames_syn)

In [100]:
df_org['syn_0'] = df_syn[[0]]
df_org['syn_1'] = df_syn[[1]]
df_org['syn_mem_status'] = df_syn[['memberstatus']]
df_org

,0,1,memberstatus,syn_0,syn_1,syn_mem_status
0,0.157652,0.842348,member,0.999905,0.000095,member
1,0.144889,0.855111,member,0.997955,0.002045,member
2,0.193260,0.806740,member,0.999169,0.000831,member
3,0.165536,0.834464,member,0.517923,0.482077,member
4,0.171518,0.828482,member,0.283809,0.716191,member
...,...,...,...,...,...,...
40,0.972808,0.027192,nonmember,0.992964,0.007036,nonmember
41,0.999956,0.000044,nonmember,0.991933,0.008067,nonmember
42,0.999977,0.000023,nonmember,0.979559,0.020441,nonmember
43,0.999996,0.000004,nonmember,0.999606,0.000394,nonmember


In [101]:
df_org.to_csv('data/prediction_comparison_org_vs_syn.csv', index=False)

In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#